In [1]:
import os
import glob
import sys
sys.path.insert(0, os.path.join(os.getcwd(), 'icenet'))
import config
from tqdm import tqdm
import pandas as pd
import xarray as xr
import time
import numpy as np

In [ ]:
# Issue: nearly 50% of the dats missing before 1991
for year in range(1979, 1978, -1):
    for month in range(12, 0, -1):
        command = 'wget -r --user="anonymous" ftp://osisaf.met.no/reprocessed/ice/conc/v2p0/' + \
                                                    str(year) + '/' + str(month).zfill(2) + '/*nh*.nc'
        print(command)
        os.system(command)

In [ ]:
# Issue: 20-21/02/2022 and 09/11/2021 missing
for year in range(2023, 2015, -1):
    for month in range(12, 0, -1):
#       command = 'wget -r --user="anonymous" ftp://osisaf.met.no/reprocessed/ice/conc-cont-reproc/v2p0/' + \
        command = 'wget -r --user="anonymous" ftp://osisaf.met.no/reprocessed/ice/conc-cont-reproc/v3p0/' + \
                                                    str(year) + '/' + str(month).zfill(2) + '/*nh*.nc'
        print(command)
        os.system(command)

In [2]:
path = '/home/jovyan/icenet-paper/icenet/osisaf.met.no/reprocessed/ice/'

The equivalent of the following cell what done with nco & cdo operators instead usign this script:
```
#!/bin/bash
for year in {1979..2015}
#for year in {2016..2023}
do
   echo "$year"
   ncrcat ice/ice_conc_nh_ease2-250_cdr-v2p0_"$year"*.nc ice_conc_nh_ease2-250_cdr-v2p0_"$year".nc
   cdo ymonmean ice_conc_nh_ease2-250_cdr-v2p0_"$year".nc ice_conc_nh_ease2-250_cdr-v2p0_"$year"_monthly_mean.nc
done
```

In [3]:
for year in range(2023, 2020, -1):
    filename = 'ice_conc_nh_ease2-250_'
    if (year >= 2016):
        filename = filename + 'i'
    filename = filename + 'cdr-v2p0_' + str(year) + '*.nc'
    files =  glob.glob(os.path.join(path, filename))
    files.sort()
    dset = xr.open_mfdataset(files).resample(time='1MS').mean()
    if (year == 2023):
        da = dset
    else:
        da = xr.concat([da, dset], dim='time')

In [4]:
da

<xarray.Dataset>
Dimensions:                   (time: 28, yc: 432, xc: 432)
Coordinates:
  * xc                        (xc) float64 -5.388e+03 -5.362e+03 ... 5.388e+03
  * yc                        (yc) float64 5.388e+03 5.362e+03 ... -5.388e+03
    lon                       (yc, xc) float32 -135.0 -135.1 ... 44.87 45.0
    lat                       (yc, xc) float32 16.62 16.82 17.02 ... 16.82 16.62
  * time                      (time) datetime64[ns] 2023-01-01 ... 2021-12-01
Data variables:
    Lambert_Azimuthal_Grid    (time) float64 -2.147e+09 ... -2.147e+09
    ice_conc                  (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    raw_ice_conc_values       (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    total_standard_error      (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    smearing_standard_error   (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    algorithm_standard_error  (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    status_flag               (time, yc, xc) float32 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>

In [ ]:
# Fix inconsistent latitude values after 2015 (names changed from icdr _> cdr)
path = '/home/jovyan/icenet-paper/icenet/osisaf.met.no/reprocessed/'
for year in range(2015, 2024, 1):
    name = 'ice_conc_nh_ease2-250_'
    if (year == 2015):
        filename = name + 'cdr-v2p0_' + str(year) + '_monthly_mean.nc'
        dset = xr.open_dataset(path + filename)
        latitudes = dset.lat.values  
    else:
        ifilename = name + 'icdr-v2p0_' + str(year) + '_monthly_mean.nc'
        dset = xr.open_dataset(path + ifilename)
        dset.lat.values = latitudes
        filename = name + 'cdr-v2p0_' + str(year) + '_monthly_mean.nc'
        dset.to_netcdf(path + filename)

In [5]:
path = '/home/jovyan/icenet-paper/icenet/osisaf.met.no/reprocessed/'
filename = 'ice_conc_nh_ease2-250_cdr-v2p0_*_monthly_mean.nc'
files =  glob.glob(os.path.join(path, filename))
files.sort()
files
da = xr.open_mfdataset(files)

In [6]:
da

<xarray.Dataset>
Dimensions:                   (time: 530, bnds: 2, yc: 432, xc: 432)
Coordinates:
  * time                      (time) datetime64[ns] 1979-01-30T12:00:00 ... 2...
    lon                       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
    lat                       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
  * xc                        (xc) float64 -5.388e+03 -5.362e+03 ... 5.388e+03
  * yc                        (yc) float64 5.388e+03 5.362e+03 ... -5.388e+03
Dimensions without coordinates: bnds
Data variables:
    time_bnds                 (time, bnds) datetime64[ns] dask.array<chunksize=(12, 2), meta=np.ndarray>
    Lambert_Azimuthal_Grid    (time) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    ice_conc                  (time, yc, xc) float64 dask.array<chunksize=(12, 432, 432), meta=np.ndarray>
    raw_ice_conc_values       (time, yc, xc) float64 dask.array<chunksize=(12, 432, 432), meta=np.ndarray>
    total_standard_error      (time, yc, xc) float64 dask.array<chunksize=(12, 432, 432), meta=np.ndarray>
    smearing_standard_error   (time, yc, xc) float64 dask.array<chunksize=(12, 432, 432), meta=np.ndarray>
    algorithm_standard_error  (time, yc, xc) float64 dask.array<chunksize=(12, 432, 432), meta=np.ndarray>
    status_flag               (time, yc, xc) float32 dask.array<chunksize=(12, 432, 432), meta=np.ndarray>
Attributes: (12/43)
    CDI:                       Climate Data Interface version 2.2.1 (https://...
    Conventions:               CF-1.6 ACDD-1.3
    source:                    FCDR of SMMR / SSMI / SSMIS Brightness Tempera...
    institution:               OSISAF - EUMETSAT
    title:                     Sea Ice Concentration Climate Data Record from...
    summary:                   This climate data record of sea ice concentrat...
    ...                        ...
    naming_authority:          int.eumetsat
    standard_name_vocabulary:  CF Standard Name Table (v36, 21 September 2016)
    id:                        10.15770/EUM_SAF_OSI_0008
    product_version:           2.0
    NCO:                       netCDF Operators version 5.1.6 (Homepage = htt...
    CDO:                       Climate Data Operators version 2.2.0 (https://...

In [7]:
### Preprocess the data
da['ice_conc'] = da['ice_conc'] / 100.  # Convert from SIC % to fraction

In [ ]:
Stopped here since I cannot find the masks

In [8]:
mask_fpath_format = os.path.join(config.mask_data_folder,
                                 config.active_grid_cell_file_format)

In [9]:
# Load grid cell mask
mask_dict = {
    month: np.load(mask_fpath_format.format('{:02d}'.format(month))) for
    month in np.arange(1, 12+1)
}

FileNotFoundError: [Errno 2] No such file or directory: 'data/masks/active_grid_cell_mask_01.npy'

In [ ]:
dates = [pd.Timestamp(date) for date in da.time.values]

In [ ]:
print('Preprocessing the SIC data:')
tic = time.time()
x = da['xc'].data
y = da['yc'].data
for date in tqdm(dates):

    # Grab mask
    mask = mask_dict[date.month]

    # Set outside mask to zero
    da.loc[date].data[~mask] = 0.

    # Grab polar hole
    skip_interp = False
    if date <= config.polarhole1_final_date:
        polarhole_mask = np.load(os.path.join(config.mask_data_folder, config.polarhole1_fname))
    elif date <= config.polarhole2_final_date:
        polarhole_mask = np.load(os.path.join(config.mask_data_folder, config.polarhole2_fname))
    elif date <= config.polarhole3_final_date and config.use_polarhole3:
        polarhole_mask = np.load(os.path.join(config.mask_data_folder, config.polarhole3_fname))
    else:
        skip_interp = True

    # Interpolate polar hole
    if not skip_interp:

        xx, yy = np.meshgrid(np.arange(432), np.arange(432))

        valid = ~polarhole_mask

        x = xx[valid]
        y = yy[valid]

        x_interp = xx[polarhole_mask]
        y_interp = yy[polarhole_mask]

        values = da.loc[date].data[valid]

        interp_vals = interpolate.griddata((x, y), values, (x_interp, y_interp), method='linear')
        interpolated_array = da.loc[date].data.copy()
        interpolated_array[polarhole_mask] = interp_vals
        da.loc[date].data = interpolated_array

In [ ]:
dur = time.time() - tic
print("Done in {}m:{:.0f}s.\n\n ".format(np.floor(dur / 60), dur % 60))

In [ ]:
fpath = os.path.join(config.obs_data_folder, 'siconca_EASE.nc')
if os.path.exists(fpath):
    os.remove(fpath)
da.to_netcdf(fpath)